In [ ]:
%matplotlib inline
import os
import h5py
import matplotlib.pyplot as plt
from ml4cvd.arguments import _get_tmap
from ml4cvd.tensor_generators import get_test_train_valid_paths

In [ ]:
tm_ecg = _get_tmap('ecg_partners')

In [ ]:
with h5py.File('/home/erik/muse_data/hd5/1980-01/000773592-1980-01-03-22.hd5', 'r') as hd5:
    ecg = tm_ecg.tensor_from_file(tm_ecg, hd5)

In [ ]:
plt.plot(ecg[11, :])

In [ ]:
tm_ecg_read = _get_tmap('ecg_partners_read')
with h5py.File('/home/erik/muse_data/hd5/1980-01/000410412-1980-01-06.hd5', 'r') as hd5:
    print(hd5['diagnosis_md'][()])
    ecg_read = tm_ecg_read.tensor_from_file(tm_ecg_read, hd5)

In [ ]:
tm_ecg_qt = _get_tmap('ecg_partners_qt_interval')
tm_ecg_intervals = _get_tmap('ecg_partners_intervals')
with h5py.File('/home/erik/muse_data/hd5/1980-01/000410412-1980-01-06.hd5', 'r') as hd5:
    ecg_qt = tm_ecg_qt.tensor_from_file(tm_ecg_qt, hd5)
    ecg_intervals = tm_ecg_intervals.tensor_from_file(tm_ecg_intervals, hd5)

In [ ]:
ecg_qt

In [ ]:
ecg_intervals